This File takes the combined NFL Career statistics, characteristics, awards, and records of NFL Quarterbacks and creates new target/y labels based on those.

The ultimate goal will be to determine which of those target/y are indicative of overall quarterback success (or can be investigated on their own) using the feature/X data from college football data.

Target/y columns:
* **hof:** Hall of Fame induction
* **wins_succ:**  wins > 50, win % > 50%
* **award_succ:** Awards total number of Pro Bowls, All Pros, MVPs, etc. eg.: ["'2x Pro Bowl'", " '1x All-Pro'"]  >= 3
* **stats_succ:** Statistical Success (passing_yards > 15,000, touchdown_passes > 50....)
* **earn_succ:** Earnings Success : career_earnings > $5M
* **long_succ:** Longevity success (years_played > 5 years)
* **metrics_succ**  wAV, AV, QBR
* **draft_succ:** Draft success: picked in 1st round, mid round, later round.

* **nfl_agg_succ:** aggregate NFL success, based on combination of the aforementioned success targets.
  * eg.: nfl_agg_succ = 1 for any player with 1 in >= 2 or more of the other columns (stats_succ =1 and wins_succ = 1).

eg.: nfl_agg_succ = 1 for any player with 1 in >= 2 or more of the other columns (stats_succ =1 and wins_succ = 1).

All columns will be 1 or 0, so ready-to-use as a target/y.  We can choose to focus on the single NFL Aggregate Sucess target, or perform branched/multi-prediction Neural Network models for multiple target/y columns.

to_do:
1) drop incorrect names (Brett Smith != Bruce Smith...)
2) check wAV values of '100' and other odd 'default' values that need to be made NaN
3) drop columns not used : 'win_record'
4) rename columns?



In [2]:
# ['player', 'draft', 'rd', 'pick', 'team', 'earn_mils', 'yrs_play',
#        'games', 'wAV', 'AV', 'win_%', 'pass_rating', 'hof', 'all_star', 'SBs',
#        'att', 'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG', 'int',
#        'int_%', 'sacks', 'pick_6', 'GWD', '4QC', 'ht', 'wt', 'hand', 'record',
#        'wins', 'loss'],

# base DataFrame
#       'player', 'draft', 'team',
#       'ht', 'wt',
# **hof:** Hall of Fame induction
#       'hof'
# **wins_succ:**  wins > 50, win % > 50%
#       'wins', 'loss', 'win_%', 
# **awards_tot, awards_succ:** Awards total number of Pro Bowls, All Pros, MVPs, etc. eg.: ["'2x Pro Bowl'", " '1x All-Pro'"]  = 3
#       'all_star',
# **stats_succ:** Statistical Success (passing_yards > 15,000, touchdown_passes > 50....)
#       'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG',       
# **earn_succ:** Earnings Success : career_earnings > $5M
#       'earn_mils',
# **long_succ:** Longevity success (years_played > 5 years)
#       'yrs_play', 'games',
# **metrics_succ**  wAV, AV, QBR
#       'wAV', 'AV', 'pass_rating',
# **draft_succ:** Draft success: picked in 1st round, mid round, later round.
# '     rd', 'pick', 
# * **nfl_agg_succ:** aggregate NFL success, based on combination of the aforementioned success targets.
#   * eg.: nfl_agg_succ = 1 for any player with 1 in >= 2 or more of the other columns (stats_succ =1 and wins_succ = 1).

# NOT USED: 'att', 'int', 'record', 'pick_6', 'hand', 'int_%', 'sacks', 

In [5]:
import pandas as pd

# <NA> used by INT64, vs. NaN
as_int = ['draft', 'rd', 'pick','yrs_play','games', 'hof', 'SBs',
       'att', 'comp', 'pass_yds', 'TD', 'pydsPG', 'int',
       'sacks', 'pick_6', 'GWD', '4QC','wt','wins', 'loss']

file_in = "../Data_Final/nfl_merged.csv"

dtype_dict = {col:'Int64' for col in as_int}

df = pd.read_csv(file_in, dtype=dtype_dict)

col_count = len(df.columns)
print(f'cols: {col_count}')

display(df.info())
display(df.columns)
display(df.iloc[:, :15].head())

if col_count >= 15:
    display(df.iloc[:, 15:].head())


cols: 34
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   player       637 non-null    object 
 1   draft        519 non-null    Int64  
 2   rd           431 non-null    Int64  
 3   pick         431 non-null    Int64  
 4   team         582 non-null    object 
 5   earn_mils    391 non-null    float64
 6   yrs_play     250 non-null    Int64  
 7   games        475 non-null    Int64  
 8   wAV          465 non-null    float64
 9   AV           396 non-null    object 
 10  win_%        370 non-null    float64
 11  pass_rating  214 non-null    float64
 12  hof          358 non-null    Int64  
 13  all_star     212 non-null    object 
 14  SBs          212 non-null    Int64  
 15  att          251 non-null    Int64  
 16  comp         250 non-null    Int64  
 17  comp_%       414 non-null    float64
 18  pass_yds     427 non-null    Int64  
 19 

None

Index(['player', 'draft', 'rd', 'pick', 'team', 'earn_mils', 'yrs_play',
       'games', 'wAV', 'AV', 'win_%', 'pass_rating', 'hof', 'all_star', 'SBs',
       'att', 'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG', 'int',
       'int_%', 'sacks', 'pick_6', 'GWD', '4QC', 'ht', 'wt', 'hand', 'record',
       'wins', 'loss'],
      dtype='object')

,player,draft,rd,pick,team,earn_mils,yrs_play,games,wAV,AV,win_%,pass_rating,hof,all_star,SBs
0,Steve Bartkowski,1975,1,1,Atlanta Falcons,NaN,12,129,100.0,84,46.46,75.4,0,['2x Pro Bowl'],0
1,Steve Beuerlein,1987,4,110,Los Angeles Raiders,NaN,16,147,100.0,85,46.08,80.3,0,['1x Pro Bowl'],1
2,Archie Manning,1971,1,2,New Orleans Saints,NaN,14,151,100.0,94,25.74,67.1,0,['2x Pro Bowl'],0
3,Brian Sipe,1972,13,330,Cleveland Browns,NaN,10,125,100.0,87,50.89,74.8,0,"['1x Pro Bowl', '1x All-Pro']",0
4,Andrew Luck,2012,1,1,Indianapolis Colts,109.108,7,86,100.0,80,61.63,89.5,0,['4x Pro Bowl'],0


,att,comp,comp_%,pass_yds,TD,YPA,pydsPG,int,int_%,sacks,pick_6,GWD,4QC,ht,wt,hand,record,wins,loss
0,3456,1932,55.9,24124,156,7.0,187,144,4.2,356,7,21,18,6-4,216,Right,59-68-0,59,68
1,3328,1894,56.9,24046,147,7.2,163,112,3.4,332,6,16,13,6-3,220,Right,47-55-0,47,55
2,3642,2011,55.2,23911,125,6.6,158,173,4.8,396,11,12,11,6-3,212,Right,35-101-3,35,101
3,3439,1944,56.5,23713,154,6.9,189,149,4.3,224,7,23,17,6-1,195,Right,57-55-0,57,55
4,3290,2000,60.8,23671,171,7.2,275,83,2.5,174,10,20,16,6-4,240,Right,53-33-0,53,33


In [7]:
# Verify no lost data:
# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

display(len(df))
# display(df[['player', 'team', 'all_star']].sort_values(by='player'))

637

### NEW Dataframe: cleaning_df

In [8]:
cleaning_df = df.copy()

display(cleaning_df['hof'].value_counts())
display(cleaning_df[(cleaning_df['hof'] == 5564) | (cleaning_df['hof'] == 1)])

hof
0       312
1         6
5564      6
1960      4
1990      3
1970      2
1982      2
1950      2
2000      2
1940      2
1963      1
1971      1
1965      1
1974      1
1978      1
1962      1
1999      1
1997      1
1987      1
1994      1
1975      1
2025      1
2010      1
1984      1
1980      1
1995      1
1966      1
Name: count, dtype: Int64

,player,draft,rd,pick,team,earn_mils,yrs_play,games,wAV,AV,win_%,pass_rating,hof,all_star,SBs,att,comp,comp_%,pass_yds,TD,YPA,pydsPG,int,int_%,sacks,pick_6,GWD,4QC,ht,wt,hand,record,wins,loss
103,Bob Waterfield,1960,7,83,Los Angeles Chargers,NaN,8,30,100.0,5,0.00,61.6,1,NaN,<NA>,1617,814,47.6,707,3,5.7,130,8,7.9,<NA>,5,6,7,6-2,184,Right,0-2-0,0,2
506,Matthew Stafford,2009,1,1,Detroit Lions,NaN,16,5,100.0,-,20.00,90.9,5564,['2x Pro Bowl'],1,7824,4952,67.4,1238,3,7.1,271,3,2.3,490,30,45,36,6-3,214,Right,1-4-0,1,4
514,Joe Flacco,2008,1,18,Baltimore Ravens,177.435,17,3,100.0,-,50.00,84.5,5564,[],1,6573,4053,65.7,716,7,6.6,234,1,2.4,392,13,26,20,6-6,230,Right,1-1-0,1,1
518,Kirk Cousins,2012,4,102,Washington Redskins,281.469,13,6,100.0,-,66.67,98.0,5564,['4x Pro Bowl'],0,5354,3584,66.7,1598,9,7.7,263,5,2.1,314,14,31,25,6-3,205,Right,4-2-0,4,2
519,Joe Montana,1999,2,49,New York Giants,NaN,16,13,100.0,NaN,NaN,92.3,1,NaN,<NA>,5391,3409,63.2,40551,273,NaN,211,139,2.6,313,6,28,26,5-10,230,NaN,NaN,<NA>,<NA>
522,Andy Dalton,2011,2,35,Cincinnati Bengals,106.514,14,5,100.0,-,25.00,87.7,5564,['3x Pro Bowl'],0,5560,3477,66.0,896,7,6.2,226,4,2.6,353,18,29,25,6-2,220,Right,1-3-0,1,3
528,Jim Kelly,1983,1,14,Buffalo Bills,NaN,11,160,100.0,132,63.12,84.4,1,"['5x Pro Bowl', '1x All-Pro']",0,4779,2874,60.1,35467,237,7.4,221,175,3.7,323,12,28,21,6-3,217,Right,101-59-0,101,59
547,Jared Goff,2016,1,1,Los Angeles Rams,233.996,9,5,100.0,-,80.00,93.6,5564,['3x Pro Bowl'],0,4231,2745,71.1,1330,8,8.9,259,4,2.0,224,9,17,14,6-4,217,Right,4-1-0,4,1
554,Patrick Mahomes,2017,1,10,Kansas City Chiefs,136.153,8,5,100.0,-,100.00,102.9,5564,"['6x Pro Bowl', '2x All-Pro']",3,3750,2497,69.4,1235,6,7.7,293,6,1.8,157,4,18,15,6-2,225,Right,5-0-0,5,0
583,Arnie Herber,<NA>,<NA>,<NA>,NaN,NaN,16,<NA>,NaN,NaN,NaN,NaN,1,NaN,<NA>,1175,<NA>,NaN,8041,81,NaN,<NA>,106,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,NaN,<NA>,<NA>


In [9]:
#  5564 are active players, set to 0
cleaning_df['hof'].replace(5564, 0, inplace=True)

hof_dict = {'Bob Waterfield': 1965,      
'Arnie Herber' : 1966,
'Benny Friedman': 2005,
'Ace Parker': 1972, 
'Joe Montana': 2000,
'Jim Kelly': 2002
}

# Update the 'hof' column based on the dictionary
cleaning_df['hof'] = cleaning_df['player'].map(hof_dict).fillna(cleaning_df['hof'])
cleaning_df['hof'].value_counts()


hof
0.0       318
1960.0      4
2000.0      3
1990.0      3
1965.0      2
1966.0      2
1982.0      2
1940.0      2
1970.0      2
1950.0      2
2005.0      1
1963.0      1
1971.0      1
1974.0      1
1978.0      1
1962.0      1
1999.0      1
1997.0      1
1987.0      1
1994.0      1
2002.0      1
1975.0      1
2025.0      1
2010.0      1
1984.0      1
1980.0      1
1995.0      1
1972.0      1
Name: count, dtype: Int64

In [10]:
# if there is a numberical value (year between 1950 and 2026) in the hof column, put a 1 in new hof_succ column in the cleaning_df dataframe

# Create the 'hof_succ' column based on the condition
cleaning_df['hof_succ'] = cleaning_df['hof'].apply(lambda x: 1 if pd.notna(x) and 1950 <= x <= 2026 else 0)

# Display the first few rows to verify the changes
display(cleaning_df[['player', 'team', 'draft','hof_succ', 'hof']].head())

,player,team,draft,hof_succ,hof
0,Steve Bartkowski,Atlanta Falcons,1975,0,0.0
1,Steve Beuerlein,Los Angeles Raiders,1987,0,0.0
2,Archie Manning,New Orleans Saints,1971,0,0.0
3,Brian Sipe,Cleveland Browns,1972,0,0.0
4,Andrew Luck,Indianapolis Colts,2012,0,0.0


In [11]:
# **wins_succ** column based on the conditions:
    # > 50 career wins
    # > 50.0% career win %

cleaning_df['wins_succ'] = ((cleaning_df['wins'] >= 32) & (cleaning_df['win_%'] > 50.0)).astype(int)

print(cleaning_df['wins_succ'].value_counts())
cleaning_df[['player', 'wins', 'win_%', 'wins_succ']].head()   

wins_succ
0    544
1     93
Name: count, dtype: int64


,player,wins,win_%,wins_succ
0,Steve Bartkowski,59,46.46,0
1,Steve Beuerlein,47,46.08,0
2,Archie Manning,35,25.74,0
3,Brian Sipe,57,50.89,1
4,Andrew Luck,53,61.63,1


In [12]:
# **awards_tot, awards_succ:** Awards total number of Pro Bowls, All Pros, MVPs, etc. 
#       'all_star'
#  return the total number of all pros, mvps, etc.; eg. ["'2x Pro Bowl'", " '1x All-Pro'"]  = 3

import numpy as np
import ast


# Function to count and sum digits in front of 'x'
def count_all_star(all_star_str):
    try:
        all_star_str = ast.literal_eval(all_star_str)
    except (ValueError, SyntaxError):
        return 0
    
    total = 0
    for item in all_star_str:
        if 'x' in item:
            try:
                count = int(item.split('x')[0])
                # print(f"Extracted count: {count} from item: {item}")
                total += count
            except ValueError:
                print(f"Skipping item: {item} due to ValueError")
                continue
    return total

cleaning_df['awards_count'] = cleaning_df['all_star'].apply(count_all_star)
cleaning_df['awards_succ'] = cleaning_df['awards_count'].apply(lambda x: 1 if x >= 2 else 0)


In [13]:
# verify all values still present
display(len(cleaning_df))
# display(df[['player', 'team', 'all_star']].sort_values(by='player').reset_index(drop=True))

637

In [14]:

display(cleaning_df[['player', 'team', 'awards_succ', 'awards_count', 'all_star']].head())

,player,team,awards_succ,awards_count,all_star
0,Steve Bartkowski,Atlanta Falcons,1,2,['2x Pro Bowl']
1,Steve Beuerlein,Los Angeles Raiders,0,1,['1x Pro Bowl']
2,Archie Manning,New Orleans Saints,1,2,['2x Pro Bowl']
3,Brian Sipe,Cleveland Browns,1,2,"['1x Pro Bowl', '1x All-Pro']"
4,Andrew Luck,Indianapolis Colts,1,4,['4x Pro Bowl']


In [15]:
#  view smaller DataFrame for statistical success; look at averages, mean, etc.

# **stats_succ:** Statistical Success (passing_yards > 15,000, touchdown_passes > 50....)
#       'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG',       

stats_cols = ['player', 'team' , 'draft', 'att' ,'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG']
stats_df = cleaning_df[stats_cols]
display(stats_df.head())

num_cols = stats_df.select_dtypes(include='number').columns.tolist() #['comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG']
display(stats_df[num_cols].describe().style.format("{:,.2f}"))


,player,team,draft,att,comp,comp_%,pass_yds,TD,YPA,pydsPG
0,Steve Bartkowski,Atlanta Falcons,1975,3456,1932,55.9,24124,156,7.0,187
1,Steve Beuerlein,Los Angeles Raiders,1987,3328,1894,56.9,24046,147,7.2,163
2,Archie Manning,New Orleans Saints,1971,3642,2011,55.2,23911,125,6.6,158
3,Brian Sipe,Cleveland Browns,1972,3439,1944,56.5,23713,154,6.9,189
4,Andrew Luck,Indianapolis Colts,2012,3290,2000,60.8,23671,171,7.2,275


,draft,att,comp,comp_%,pass_yds,TD,YPA,pydsPG
count,519.00,251.00,250.00,414.00,427.00,416.00,399.00,214.00
mean,"1,998.09","3,159.55","1,847.30",56.88,"13,134.34",85.82,6.68,179.72
std,22.38,"1,876.07","1,198.76",8.62,"14,243.82",95.52,1.61,45.73
min,"1,937.00","1,110.00",591.00,0.00,5.00,1.00,0.00,71.00
25%,"1,982.50","1,736.50",969.75,52.42,"2,281.00",16.00,6.20,148.25
50%,"2,005.00","2,634.00","1,535.50",56.75,"7,975.00",49.50,6.70,176.00
75%,"2,017.00","4,006.00","2,303.25",60.58,"20,506.50",128.25,7.10,210.00
max,"2,024.00","12,050.00","7,753.00",100.00,"89,214.00",649.00,24.00,293.00


In [16]:
# View a dataframe with the determined criteria.

criteria = {
    'comp': 1200,       # or more career passing completions
    'comp_%': 70,       # or more career completion percent : comp %
    'pass_yds': 15000,  # or more passing yards
    'TD': 75,           # or more passing touchdowns
    'YPA': 7.0,         # or more Yards Per pass Attempt
    'pydsPG': 200       # or more pass Yards Per Game
}

stats_df = stats_df[
    (stats_df['comp'] > criteria['comp']) |
    (df['comp_%'] >= criteria['comp_%']) |
    (df['pass_yds'] >= criteria['pass_yds']) |
    (df['TD'] >= criteria['TD']) |
    (df['YPA'] >= criteria['YPA']) |
    (df['pydsPG'] >= criteria['pydsPG'])          
    ]

display(len(stats_df))
# display(stats_df)


227

In [17]:
# quick dataclean for comp_%  anything over 90% is wrong.

import numpy as np
cleaning_df['comp_%'] = cleaning_df['comp_%'].apply(lambda x: np.nan if x >= 90 else x)
cleaning_df[cleaning_df['comp_%'] >= 90 ]

,player,draft,rd,pick,team,earn_mils,yrs_play,games,wAV,AV,win_%,pass_rating,hof,all_star,SBs,att,comp,comp_%,pass_yds,TD,YPA,pydsPG,int,int_%,sacks,pick_6,GWD,4QC,ht,wt,hand,record,wins,loss,hof_succ,wins_succ,awards_count,awards_succ


In [18]:
# Now apply the above criteria for the Stats success column, and add it to the cleaning dataframe
import numpy as np

# Assuming df and criteria are already defined
cleaning_df.fillna(0, inplace=True)

# Update the logic to handle NaN values
cleaning_df['stats_succ'] = (
    (cleaning_df['comp'].fillna(0) >= criteria['comp']) |
    (cleaning_df['comp_%'].fillna(0) >= criteria['comp_%']) |
    (cleaning_df['pass_yds'].fillna(0) >= criteria['pass_yds']) |
    (cleaning_df['TD'].fillna(0) >= criteria['TD']) |
    (cleaning_df['YPA'].fillna(0) >= criteria['YPA']) |
    (cleaning_df['pydsPG'].fillna(0) >= criteria['pydsPG'])
).astype(np.int64)

display(len(cleaning_df))
stats_cols_updated = ['player', 'team' , 'draft', 'stats_succ', 'att' ,'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG']
display(cleaning_df[stats_cols_updated].head())
display(len(cleaning_df['stats_succ'][cleaning_df['stats_succ'] == 1]))
cleaning_df['stats_succ'].value_counts()

637

,player,team,draft,stats_succ,att,comp,comp_%,pass_yds,TD,YPA,pydsPG
0,Steve Bartkowski,Atlanta Falcons,1975,1,3456,1932,55.9,24124,156,7.0,187
1,Steve Beuerlein,Los Angeles Raiders,1987,1,3328,1894,56.9,24046,147,7.2,163
2,Archie Manning,New Orleans Saints,1971,1,3642,2011,55.2,23911,125,6.6,158
3,Brian Sipe,Cleveland Browns,1972,1,3439,1944,56.5,23713,154,6.9,189
4,Andrew Luck,Indianapolis Colts,2012,1,3290,2000,60.8,23671,171,7.2,275


228

stats_succ
0    409
1    228
Name: count, dtype: int64

In [19]:
# **earn_succ:** Earnings Success : career_earnings > $5M
#       'earn_mils',
cleaning_df['earn_succ'] = (cleaning_df['earn_mils'] >= 1).astype(int)

earn_df = cleaning_df[['player', 'team', 'draft', 'earn_succ','earn_mils']]


print(earn_df['earn_succ'].value_counts())
earn_df[['player', 'team', 'draft', 'earn_succ','earn_mils']].head()   


earn_succ
0    410
1    227
Name: count, dtype: int64


,player,team,draft,earn_succ,earn_mils
0,Steve Bartkowski,Atlanta Falcons,1975,0,0.000
1,Steve Beuerlein,Los Angeles Raiders,1987,0,0.000
2,Archie Manning,New Orleans Saints,1971,0,0.000
3,Brian Sipe,Cleveland Browns,1972,0,0.000
4,Andrew Luck,Indianapolis Colts,2012,1,109.108


In [20]:
# **long_succ:** Longevity success (years_played > 5 years)
#       'yrs_play', 'games',

# long_df = cleaning_df['long_succ']
long_df = cleaning_df[['player', 'team', 'draft', 'games','yrs_play']]
long_df.describe()

cleaning_df['long_succ'] = ((cleaning_df['yrs_play'] >= 4) & (cleaning_df['games'] >= 32)).astype(int)

print(cleaning_df['long_succ'].value_counts())
cleaning_df[['player', 'team', 'draft', 'long_succ', 'yrs_play', 'games']].head()

long_succ
0    420
1    217
Name: count, dtype: int64


,player,team,draft,long_succ,yrs_play,games
0,Steve Bartkowski,Atlanta Falcons,1975,1,12,129
1,Steve Beuerlein,Los Angeles Raiders,1987,1,16,147
2,Archie Manning,New Orleans Saints,1971,1,14,151
3,Brian Sipe,Cleveland Browns,1972,1,10,125
4,Andrew Luck,Indianapolis Colts,2012,1,7,86


In [26]:
# **metrics_succ**  wAV, AV, QBR
#       'wAV', 'AV', 'pass_rating',

cleaning_df['AV'] = pd.to_numeric(cleaning_df['AV'], errors='coerce')
# Fill NaN values with 0
cleaning_df['AV'].fillna(0, inplace=True)
cleaning_df['AV'] = cleaning_df['AV'].astype(float)

metrics_df = cleaning_df[['player', 'team', 'draft', 'wAV','AV', "pass_rating"]]
display(metrics_df.describe())

cleaning_df['metrics_succ'] = ((cleaning_df['wAV'] >= 70) | 
                            (cleaning_df['AV'] >= 10) |
                            (cleaning_df['pass_rating'] >= 100)       
                               ).astype(int)


display(cleaning_df['metrics_succ'].value_counts())
display(cleaning_df[['player', 'team', 'draft', 'metrics_succ', 'wAV', 'AV', 'pass_rating']].head())


# sample AV:  Colt McCoy: 7 (in 2011), Joe Flacco 13 (in 2014), Jake Delhomme: 12 (2004)
# sample wAV: Colt McCoy: 22         , Joe Flacco: 76/95         , Jake Delhomme: 55 
# The average career passer rating for an NFL starter typically falls around 85-90. Modern quarterbacks tend to have
#  higher passer ratings due to changes in the game that favor passing offenses

cleaning_df.columns


,draft,wAV,AV,pass_rating
count,637.0,637.000000,637.000000,637.000000
mean,1627.956044,72.998430,27.601256,26.514443
std,777.119614,44.431648,48.102497,37.765802
min,0.0,0.000000,-4.000000,0.000000
25%,1959.0,0.000000,0.000000,0.000000
50%,1995.0,100.000000,1.000000,0.000000
75%,2015.0,100.000000,35.000000,72.300000
max,2024.0,100.000000,326.000000,103.000000


metrics_succ
1    465
0    172
Name: count, dtype: int64

,player,team,draft,metrics_succ,wAV,AV,pass_rating
0,Steve Bartkowski,Atlanta Falcons,1975,1,100.0,84.0,75.4
1,Steve Beuerlein,Los Angeles Raiders,1987,1,100.0,85.0,80.3
2,Archie Manning,New Orleans Saints,1971,1,100.0,94.0,67.1
3,Brian Sipe,Cleveland Browns,1972,1,100.0,87.0,74.8
4,Andrew Luck,Indianapolis Colts,2012,1,100.0,80.0,89.5


Index(['player', 'draft', 'rd', 'pick', 'team', 'earn_mils', 'yrs_play',
       'games', 'wAV', 'AV', 'win_%', 'pass_rating', 'hof', 'all_star', 'SBs',
       'att', 'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG', 'int',
       'int_%', 'sacks', 'pick_6', 'GWD', '4QC', 'ht', 'wt', 'hand', 'record',
       'wins', 'loss', 'hof_succ', 'wins_succ', 'awards_count', 'awards_succ',
       'stats_succ', 'earn_succ', 'long_succ', 'metrics_succ'],
      dtype='object')

In [27]:

# * **nfl_agg_succ:** aggregate NFL success, based on combination of the aforementioned success targets.
#   * eg.: nfl_agg_succ = 1 for any player with 1 in >= 2 or more of the other columns (stats_succ =1 and wins_succ = 1).


# Display the number of rows in cleaning_df
print(len(cleaning_df))

# Create nfl_succ_df with the specified columns
nfl_succ_df = cleaning_df[['player', 'team', 'draft', 'hof_succ', 'wins_succ', 'awards_succ', 'stats_succ', 'earn_succ', 'long_succ', 'metrics_succ']]
numeric_columns = ['hof_succ', 'wins_succ', 'awards_succ', 'stats_succ', 'earn_succ', 'long_succ', 'metrics_succ']


# NFL success defined as having three or more of the various _succ categories: 
nfl_succ_df['nfl_success'] = (nfl_succ_df[numeric_columns].sum(axis=1) >= 3).astype(int)

# Move 'nfl_success' column right after 'draft' column
cols = list(nfl_succ_df.columns)
draft_index = cols.index('draft')
cols.insert(draft_index + 1, cols.pop(cols.index('nfl_success')))
nfl_succ_df = nfl_succ_df[cols]

# Sort by 'nfl_success' and then by 'player'
nfl_succ_df.sort_values(by=['nfl_success', 'player'], ascending=[False, True], inplace=True)
nfl_succ_df.reset_index(drop=True, inplace=True)

display(nfl_succ_df['nfl_success'].value_counts())
display(nfl_succ_df.head())

637


C:\Users\black\AppData\Local\Temp\ipykernel_19180\1593570963.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nfl_succ_df['nfl_success'] = (nfl_succ_df[numeric_columns].sum(axis=1) >= 3).astype(int)


nfl_success
0    402
1    235
Name: count, dtype: int64

,player,team,draft,nfl_success,hof_succ,wins_succ,awards_succ,stats_succ,earn_succ,long_succ,metrics_succ
0,Aaron Brooks,Green Bay Packers,1999,1,0,0,0,1,0,1,1
1,Aaron Rodgers,Green Bay Packers,2005,1,1,0,1,1,1,0,1
2,Al Dorow,Washington Redskins,1952,1,0,0,1,0,0,1,1
3,Alex Smith,Buffalo Bills,2005,1,0,0,0,1,1,0,1
4,Alex Tanney,Undrafted,2015,1,0,0,0,1,1,0,1


In [28]:
#Export csv of JUST the target/y labels:
file_out = "../Data_Final/nfl_success_labels.csv"
nfl_succ_df.to_csv(file_out, index=False)
print(f'saved csv to: {file_out}')

saved csv to: ../Data_Final/nfl_success_labels.csv


In [29]:
file_out = "../Data_Final/nfl_success_full_dataframe.csv"
cleaning_df.to_csv(file_out, index=False)
print(f'saved csv to: {file_out}')

saved csv to: ../Data_Final/nfl_success_full_dataframe.csv


# THESE CELLS NOT USED

In [25]:
# base_cols = ['player', 'team', 'draft']
# hof_cols = ['hof']
# wins_cols = ['win_succ', 'wins', 'loss', 'win_%']
# awards_cols = ['awards_succ', 'awards_count', 'all_star']
# stats_cols = ['stats_succ' , 'comp', 'comp_%', 'pass_yds', 'TD', 'YPA', 'pydsPG']
# earn_cols = ['earn_succ', 'earn_mils' ]
# long_cols= ['long_succ', 'yrs_play', 'games']
# metrics_cols = ['metrics_succ' ,'wAV', 'AV', 'pass_rating']
# draft_cols =  ['draft_succ', 'rd', 'pick']